# Hello cả nhà, tớ xin thề, đây sẽ là model cuối cùng của tớ :))))))

Import libraries, dependencies

In [1]:
import numpy as np
np.random.seed(5) 
import tensorflow as tf
tf.random.set_seed(2)
import matplotlib.pyplot as plt
%matplotlib inline
import os
import cv2
import keras

## Loading training dataset

In [2]:
train = (r"G:\Handsign model\Dataset\aslmini")
test = (r"G:\Handsign model\Dataset\asl7gb\Test_Alphabet")

In [3]:
def load_images(directory):
    images = []
    labels = []
    for idx, label in enumerate(characters):
        for file in os.listdir(directory + "/" + label):
            filepath = directory + "/" + label + "/" + file
            image = cv2.imread(filepath)
            images.append(image)
            labels.append(idx)
    images = np.array(images)
    labels = np.array(labels)
    return(images, labels)

In [4]:
characters = sorted(os.listdir(train))
images, labels = load_images(train)

In [5]:
images.shape

(866, 200, 200, 3)

## Image augmentation

In [6]:
import albumentations as A

In [7]:
transform = A.Compose([
    A.RandomCrop(height=100, width=100, p=0.7),
    A.RandomBrightnessContrast(brightness_limit=(-0.64, 0.54), contrast_limit=(-0.69, 0.64), p=0.4),
    A.Rotate(limit=(-43, 46), interpolation=0, p=0.4),
    A.Blur(p=0.3),
    # A.Resize(height=200, width=200, p=1)
])

In [8]:
def image_augmentation(originate):
    augmented_image = transform(image=originate)['image']
    new_image = np.concatenate((originate, augmented_image))
    return new_image

x3 image augmentation

## Model design

In [9]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import MobileNetV2
from keras import layers

In [10]:
resnet = ResNet50(
    include_top=False, 
    weights=None, 
    input_tensor= layers.Input(shape=(200, 200, 3)), 
    input_shape = (200, 200, 3),
    pooling='avg',
    )

In [11]:
mobilenetv2 = MobileNetV2(
    include_top=False,
    weights=None, 
    input_tensor= layers.Input(shape=(200, 200, 3)), 
    input_shape = (200, 200, 3),
    pooling='avg',
)

In [12]:
image = cv2.imread("test.png")
image = cv2.resize(image, (200, 200))
image = np.expand_dims(image, axis=0)

### live video feed 

In [13]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Unable to open webcam")
    exit()

In [14]:
# cap = cv2.VideoCapture(0)
# if not cap.isOpened():
#     print("Camera can not be opened")
# while True:
#     ret, frame = cap.read()
#     if not ret:
#         print("video capture failed")
#         break
#     image = cv2.resize(frame, (512, 512))
#     image = np.expand_dims(image, axis=0)
#     result = model.predict(image)
#     cv2.imshow("CV2 video feed", frame)
#     cv2.imshow("CV2 video feed", frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
# cap.release()
# cv2.destroyAllWindows()

In [15]:
result = resnet.predict(image)

1/1 [==============================] - 2s 2s/step


In [16]:
print(result.shape)

(1, 2048)


### Model designing

In [17]:
model = keras.Sequential()

In [18]:
model.add(resnet)
model.add(layers.Dense(64, activation="relu", input_shape=(2048,)))
model.add(layers.Dense(29, activation="softmax"))

In [19]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 64)                131136    
                                                                 
 dense_1 (Dense)             (None, 29)                1885      
                                                                 
Total params: 23,720,733
Trainable params: 23,667,613
Non-trainable params: 53,120
_________________________________________________________________


## Start training

In [21]:
from sklearn.model_selection import train_test_split

Split train, test, valid data

In [22]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size = 0.1, stratify = labels)

In [23]:
result = model.predict(np.expand_dims(X_train[0], axis=0))

1/1 [==============================] - 1s 512ms/step


In [24]:
y_train.shape

(779,)

In [25]:

n = len(characters)
train_n = len(X_train)
test_n = len(X_test)

print("Total number of symbols: ", n)
print("Number of training images: " , train_n)
print("Number of testing images: ", test_n)

eval_n = len(X_test)
print("Number of evaluation images: ", eval_n)

Total number of symbols:  29
Number of training images:  779
Number of testing images:  87
Number of evaluation images:  87


Preprocessing

In [26]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [27]:
X_train = X_train.astype('float32')/255.0
X_test = X_test.astype('float32')/255.0

Training model thôi trời đất ơiiiii

In [30]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
callbacks = [ ReduceLROnPlateau(monitor='loss', patience=2, cooldown=0),
              EarlyStopping(monitor='accuracy', min_delta=1e-4, patience=2)]

In [31]:
hist = model.fit(X_train, y_train, epochs = 100, batch_size = 64,callbacks=callbacks)

Epoch 1/100
13/13 [==============================] - 5s 367ms/step - loss: 2.9993 - accuracy: 0.1374 - lr: 0.0010
Epoch 2/100
13/13 [==============================] - 5s 373ms/step - loss: 2.9016 - accuracy: 0.1348 - lr: 0.0010
Epoch 3/100
13/13 [==============================] - 5s 370ms/step - loss: 2.8520 - accuracy: 0.1463 - lr: 0.0010
Epoch 4/100
13/13 [==============================] - 5s 370ms/step - loss: 2.8351 - accuracy: 0.1720 - lr: 0.0010
Epoch 5/100
13/13 [==============================] - 5s 370ms/step - loss: 2.7758 - accuracy: 0.1425 - lr: 0.0010
Epoch 6/100
13/13 [==============================] - 5s 370ms/step - loss: 2.6789 - accuracy: 0.1759 - lr: 0.0010
Epoch 7/100
13/13 [==============================] - 5s 371ms/step - loss: 2.6428 - accuracy: 0.1784 - lr: 0.0010
Epoch 8/100
13/13 [==============================] - 5s 371ms/step - loss: 2.7141 - accuracy: 0.1772 - lr: 0.0010
Epoch 9/100
13/13 [==============================] - 5s 371ms/step - loss: 2.6651 - accu